# 이미지를 통한 얼굴 인식
1. 시스템에 입력된 이미지에서 얼굴 영역을 찾는다.(얼굴 검출, face detection)
2. 눈, 코, 입 등 얼굴의 특징점을 찾는다.(얼굴 정렬, face alignment)
3. 위 특징들을 통해 얼굴 영역을 동일한 형태와 크기로 변경(정규화, normalization)

# 영상을 통한 얼굴 인식


영상을 통한 표정 인식

In [1]:

!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2  908k    2 19194    0     0  47985      0  0:00:19 --:--:--  0:00:19 47865
 51  908k   51  464k    0     0   320k      0  0:00:02  0:00:01  0:00:01  320k
 69  908k   69  630k    0     0   261k      0  0:00:03  0:00:02  0:00:01  261k
 85  908k   85  776k    0     0   227k      0  0:00:03  0:00:03 --:--:--  227k
 97  908k   97  888k    0     0   196k      0  0:00:04  0:00:04 --:--:--  196k
100  908k  100  908k    0     0   200k      0  0:00:04  0:00:04 --:--:--  215k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3  878k    3 34137    0     0  38485      0  0:0

In [2]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model


In [7]:
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile = False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

# camera객체 생성
camera = cv2.VideoCapture(0)

while True:
    # camera에서 이미지 캡쳐
    ret, frame = camera.read()
    # 이미지의 색상을 흑백으로 변환
    # cv2.cvtColor(원본 이미지, 색상 변환 코드)
    # HSV : Hue, Saturation, Value -> 색상, 채도, 명도
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    faces = face_detection.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
                                           minSize = (30, 30))
    canvas = np.zeros((250, 300, 3), dtype = 'uint8')
    
    if len(faces) > 0:
        face = sorted(faces, reverse = True, key = lambda x: (x[2] - x[0]) * 
                     (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
        
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = '{} : {:.2f}%'.formate(emotion, prob * 100)
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
            
        cv2.imshow('Emotion Recognition', frame)
        cv2.imshow('Probabilities', canvas)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
camera.release()
cv2.destroyAllWindows()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


이미지를 통한 얼굴 인식
- 소프트맥스 손실 함수(sortmax loss function)

> 1) 교차 엔트로피(cross entropy)가 소프트 맥스 확률 분포와 정답 분포 사이의 오차를 계산

> 단점 : (1) 많은 데이터의 특징 공간을 효율적으로 학습하기 어렵다. (2) 전체적인 최적화를 고려하지 못하고 국부적인 최적 지점으로 쉽게 수렴할 수 있다. (3) 학습하지 않은 새로운 얼굴 이미지의 인식 성능이 낮아질 수 있다.

- 거리 기반 손실 함수(distance-based loss function)

> 대표적인 거리 기반 손실 함수 : 대비(contrastive), 트리플렛(triplet)

> 대비 손실 함수 : 두 얼굴 이미지의 쌍을 구성해 두 특징 벡터 간의 거리를 계산한다. 손실 값은 동일인의 두 벡터 간 거리가 멀거나, 비동일인의 두 벡터간 거리가 가까우면 커진다.

> 대비 손실 함수의 단점 : 동일인에 해당하는 두 벡터 거리와 비동일인의 두 벡터 거리가 개별적으로 학습된다.

> 트리플렛 손실 함수


- 앵귤러 마진 기반 손실 함수(angular margin based loss function)


''' https://tech.kakaoenterprise.com/63 '''

In [ ]:
'''참고 자료'''


'''
https://m.blog.naver.com/roboholic84/221633210887 - 딥러닝으로 표정 인식하기 알고리즘
https://tech.kakaoenterprise.com/63 - 얼굴 인식 알고리즘 선행 연구
http://www.kibme.org/resources/journal/20180801145420149.pdf - (논문) 딥러닝 기반 얼굴 검출,
랜드마크 검출 및 얼굴 인식 기술 연구 동향
https://openaccess.thecvf.com/content_CVPR_2020/papers/Kim_GroupFace_Learning_Latent_Groups_and_Constructing_Group-Based_Representations_for_Face_CVPR_2020_paper.pdf - AI Lab 논문




'''